In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import re
from webdriver_manager.chrome import ChromeDriverManager
import time


In [2]:
def requestPage(link):
    try:
        response = requests.get(link)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        return response.text  # Return the HTML content of the page
    except requests.RequestException as e:
        print(f"Error fetching page: {e}")
        return None


In [3]:
def requestPageUsingWebDriver(self, link, web_agent="chrome"):
   options = driver = None
   if web_agent == "firefox":
       options = webdriver.FirefoxOptions()
       options.add_argument("--headless")
       driver = webdriver.Firefox(options=options)
   elif web_agent == "chrome":
       options = Options()
       options.add_argument("--disable-dev-shm-usage")
       options.add_argument("--headless")
       options.add_argument("--no-sandbox")
       driver = webdriver.Chrome(options=options)
   else:
       raise Exception("Invalid Web Agent! Please use firefox or chrome.")
   driver.get(link)
   driver.implicitly_wait(50)
   soup = BeautifulSoup(driver.page_source, "html.parser")
   driver.quit()
   return soup



In [4]:
def requestPage(link):
    try:
        print(f"Fetching page: {link}")
        response = requests.get(link)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        soup = BeautifulSoup(response.text, 'html.parser')  # Parse the HTML content
        return soup
    except requests.RequestException as e:
        print(f"Error fetching page: {e}")
        return None


In [17]:
def extractArticleLinks(category_link, show_more,max_links=7000,sections=['news']):
    article_links = []
    domain_url = 'https://www.aljazeera.com/tag/israel-palestine-conflict/'
    news_regex = re.compile(r'\/news\/\d+')
    
    options = Options()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
    wait = WebDriverWait(driver, 2)  # Adjust the timeout as needed
    
    driver.get(category_link)
    for i in range(show_more):
        try:
            time.sleep(3)
            show_more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "show-more-button.big-margin")))
                        
            #show_more_button = driver.find_element(By.CLASS_NAME, "screen-reader-text")
            driver.execute_script("arguments[0].click();", show_more_button)
        except Exception as e:
            print("No more 'Show More' buttons to click.")
            break
    
    while len(article_links) < max_links:
        article_tags = driver.find_elements(By.CSS_SELECTOR, "a.u-clickable-card__link")
        for a_tag in article_tags:
            link = a_tag.get_attribute("href")
            if any(section in link for section in sections):
                if not link.startswith("http"):
                    link = domain_url + link
                if news_regex.search(link):
                    article_links.append(link)
                    if len(article_links) >= max_links:
                        break


    driver.quit()  # Close the WebDriver session
    
    return article_links[:max_links]  # Return only up to max_links
